# Apprentissage supervisé

Le but de ce TP est d'implémenter et tester les algorithmes vues en classes.

## Utilitaire

Ci-dessous des fonctions à utiliser pour le TP.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np


def plot_points(points, ax, color):
    """
        Plots points.
        :param points: the points to plot
        :type points: iterable
        :param ax: the axis to plot in
        :type ax: matplotlib.axis
        :param color: the plotted points color
        :type color: str
    """
    ax.scatter(points[:, 0], points[:, 1], c=color)

    
def plot_dataset(X, Y, ax, **parameters):
    """
        Plots the dataset.
        :param X: the points to plot
        :type X: np.array
        :param Y: the points classes
        :type Y: np.array
        :param ax: the axis to plot in
        :type ax: matplotlib.axis
        :param colors: the plotted points colors depending on the class
        :type colors: iterable
    """
    for x, color in zip([X[Y==y] for y in set(Y)], parameters['colors']):
        plot_points(x, ax, color)

        
def mesh_from(X, gap=.2):
    """
        Constructs mesh points from some 2D instances
        :param X: the 2D instances
        :type X: np.array
        :param gap: the gap between two mesh points
        :type gap: float
        :return: the mesh points
        :rtype: tuple
    """
    return np.meshgrid(
        np.arange(X[:, 0].min() - 1, X[:, 0].max() + 1, gap),
        np.arange(X[:, 1].min() - 1, X[:, 1].max() + 1, gap),
    )


def plot_separator(xx, yy, ax, classifier, **parameters):
    """
        Plots separator.
        
        :param xx: mesh first coordinates
        :type xx: np.array
        :param yy: mesh second coordinates
        :type yy: np.array
        :param ax: subplot to draw in
        :type ax: matplotlib.axis
        :param classifier: the trained classifier
        :type classifier: sklearn.base.ClassifierMixin
    """
    Z = classifier.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z, **parameters)
